In [1]:
from OpenRefineClientPy3.google_refine.refine import refine
from OR import Refineop
import pandas as pd
import json
from pprint import pprint

# Project

In [5]:
# list project
projects = refine.Refine(refine.RefineServer()).list_projects()
print(type(projects))
pprint(projects)

<class 'dict'>
{'1596601650071': {'contributors': '',
                   'created': '2020-04-14T20:32:24Z',
                   'creator': '',
                   'customMetadata': {},
                   'description': '',
                   'homepage': '',
                   'image': '',
                   'importOptionMetadata': [{'encoding': '',
                                             'fileSource': 'CoronaVirus_tweets.csv',
                                             'guessCellValueTypes': False,
                                             'headerLines': 1,
                                             'ignoreLines': -1,
                                             'includeFileSources': False,
                                             'limit': -1,
                                             'processQuotes': True,
                                             'projectName': 'tweet',
                                             'projectTags': [''],
                             

In [6]:
server = refine.RefineServer()
projectID = 1596601650071
refineop = Refineop(server,projectID)

# list history (From internal package)
# each operation changes will be assigned with a history id
history_id = refineop.list_history()
pprint(history_id)
hid = [hid['id'] for hid in history_id['past']]
print(hid)

{'future': [{'description': 'Text transform on 1000 cells in column Author: '
                            'value.trim()',
             'id': 1586988418985,
             'time': '2020-04-15T22:06:02Z'},
            {'description': 'Create new column Author_Name based on column '
                            'Author by filling 1000 rows with '
                            "grel:cells.Author.value+'-'+cells.Name.value",
             'id': 1586988520301,
             'time': '2020-04-15T22:06:02Z'},
            {'description': 'Text transform on 991 cells in column '
                            'Author_Name: value.toLowercase()',
             'id': 1586988853204,
             'time': '2020-04-15T22:06:02Z'}],
 'past': [{'description': 'Text transform on 1000 cells in column Date (CST): '
                          'value.toDate()',
           'id': 1586988827743,
           'time': '2020-04-15T22:06:02Z'},
          {'description': 'Text transform on 1000 cells in column Author: '
           

In [8]:
# redo step 3: hid == 
refineop.undo_proj(lastDone_id = 1586988418985)

True

In [9]:
history_id = refineop.list_history()
pprint(history_id)

{'future': [{'description': 'Create new column Author_Name based on column '
                            'Author by filling 1000 rows with '
                            "grel:cells.Author.value+'-'+cells.Name.value",
             'id': 1586988520301,
             'time': '2020-04-15T22:06:02Z'},
            {'description': 'Text transform on 991 cells in column '
                            'Author_Name: value.toLowercase()',
             'id': 1586988853204,
             'time': '2020-04-15T22:06:02Z'}],
 'past': [{'description': 'Text transform on 1000 cells in column Date (CST): '
                          'value.toDate()',
           'id': 1586988827743,
           'time': '2020-04-15T22:06:02Z'},
          {'description': 'Text transform on 1000 cells in column Author: '
                          'value.replace("@"," ")',
           'id': 1586988628991,
           'time': '2020-04-15T22:06:02Z'},
          {'description': 'Text transform on 1000 cells in column Author: '
         

In [10]:
# list operation history 
operation_history = refineop.get_operations()
pprint(operation_history)

[{'columnName': 'Date (CST)',
  'description': 'Text transform on cells in column Date (CST) using '
                 'expression value.toDate()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.toDate()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.replace("@"," ")',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.replace("@"," ")',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.trim()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.trim()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCou

# Cell

In [11]:
# text transform : column name, expression
refineop.text_transform('Author',expression = 'value.toUppercase()')

{'code': 'ok',
 'historyEntry': {'id': 1586988764246,
  'description': 'Text transform on 976 cells in column Author: value.toUppercase()',
  'time': '2020-04-15T22:06:33Z'}}

In [12]:
from collections import OrderedDict
# cluter and Edit 
# 1. compute clusters: return a list of clusters; 
#    params: column, clusterer_type='binning', function=None, params=None
# 2. edits 
# mass_edit
clusters = refineop.compute_clusters('Country', clusterer_type='knn', function='levenshtein')
print(clusters)

# choose cluster
from_edit=refineop.getFromValue(clusters)
to_edit=refineop.getToValue(clusters)
default_edits=OrderedDict()
default_edits['fromBlank']='false'
default_edits['fromError']='false'
for f1,t1 in zip(from_edit, to_edit):
    default_edits['from']=f1
    default_edits['to']=t1
edits=[default_edits]
# mass edit : column, edits
refineop.mass_edit('Country', edits)

[[{'value': 'United States of America', 'count': 187}, {'value': 'United States of @merica', 'count': 5}]]


{'code': 'ok',
 'historyEntry': {'id': 1586989099980,
  'description': 'Mass edit 192 cells in column Country',
  'time': '2020-04-15T22:06:34Z'}}

In [13]:
pprint(refineop.get_operations())

[{'columnName': 'Date (CST)',
  'description': 'Text transform on cells in column Date (CST) using '
                 'expression value.toDate()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.toDate()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.replace("@"," ")',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.replace("@"," ")',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCount': 10},
 {'columnName': 'Author',
  'description': 'Text transform on cells in column Author using expression '
                 'value.trim()',
  'engineConfig': {'facets': [], 'mode': 'row-based'},
  'expression': 'value.trim()',
  'onError': 'keep-original',
  'op': 'core/text-transform',
  'repeat': False,
  'repeatCou

In [14]:
# single edit (27,5) from (@KIMBA707) to (KIMBA707)
# params: row, cell, type, value
# is there any ops recorded?
refineop.single_edit(27,5,'text','KIMBA707')

{'code': 'ok',
 'historyEntry': {'id': 1586988597880,
  'description': 'Edit single cell on row 28, column Name',
  'time': '2020-04-15T22:06:36Z'},
 'cell': {'v': 'KIMBA707'},
 'pool': {'recons': {}}}

# Column

In [15]:
# addition column
# params: column, new_column, expression='value', column_insert_index=None
# addition expression could be found https://github.com/OpenRefine/OpenRefine/wiki/Column-Editing
refineop.add_column('Country','Country_State', expression='cells.Country.value+cells.State.value')

{'code': 'ok',
 'historyEntry': {'id': 1586988963126,
  'description': 'Create new column Country_State based on column Country by filling 0 rows with cells.Country.value+cells.State.value',
  'time': '2020-04-15T22:06:37Z'}}

In [16]:
# split column
# params: column, separator=','
refineop.split_column('Name',' ')

{'code': 'ok',
 'historyEntry': {'id': 1586989274756,
  'description': 'Split 1000 cell(s) in column Name into several columns by separator',
  'time': '2020-04-15T22:06:38Z'}}

In [17]:
# rename column
# params: old column name, new column name
refineop.rename_column('Name 1', 'FirstName')

{'code': 'ok',
 'historyEntry': {'id': 1586988458941,
  'description': 'Rename column Name 1 to FirstName',
  'time': '2020-04-15T22:06:40Z'}}

# Row level

In [20]:
# Annotate row (star/flag)
# row index, flagged:default='True'
refineop.flag_row(1)
refineop.star_row(2)

{'code': 'ok',
 'historyEntry': {'id': 1586988552080,
  'description': 'Star row 3',
  'time': '2020-04-15T22:07:16Z'}}

In [ ]:
# apply operation history